In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
X = np.genfromtxt("X_train.csv", delimiter=',', dtype=str)[1:]
Y = np.genfromtxt("Y_train.csv", delimiter=',', dtype=float)[1:, 1]
odometer = copy.copy(X[:, 5]).astype('int64')
engineCapacity = np.genfromtxt("X_train.csv", delimiter=',',
                               dtype=float)[1:, 9]
photos = copy.copy(X[:, 12]).astype('int64')
features = np.genfromtxt("X_train.csv", delimiter=',', dtype=bool)[1:, 13:]
num_features = np.sum(features, axis=1)
X = np.array(
    [np.ones(X.shape[0]), odometer, engineCapacity, photos, num_features, Y]).T
X = X[~np.isnan(X).any(axis=1)]
Y = copy.copy(X[:, 5:])
Y = Y.reshape((Y.shape[0], 1))
X = np.delete(X, 4, 1)

Transform $X \rightarrow Z = (1, x_1^2, x_2^2, x_3^2, x_4^2, x_1x_2, x_2x_3, x_3x_4, x_4x_1)$

In [3]:
def trans1(X):
    Z = np.array([
        X[:, 0],
        X[:, 1]**2,
        X[:, 2]**2,
        X[:, 3]**2,
        X[:, 4]**2,
        X[:, 1] * X[:, 2],
        X[:, 2] * X[:, 3],
        X[:, 3] * X[:, 4],
        X[:, 4] * X[:, 1],
    ]).T
    return Z

In [4]:
Z1 = trans1(X)

In [5]:
def linear_regression(Z, Y):
    return np.linalg.pinv(Z) @ Y

In [6]:
def avg_err(Z, Y):
    avg_err_in = 0
    avg_err_out = 0
    for _ in range (10000):
        Z_train, Z_test, Y_train, Y_test = train_test_split(Z, Y, shuffle=True)
        w = linear_regression(Z_train, Y_train)
        avg_err_in += np.mean(np.abs(Y_train - Z_train @ w)) / 10000
        avg_err_out += np.mean(np.abs(Y_test - Z_test @ w)) / 10000
    return avg_err_in, avg_err_out

In [7]:
avg_err_in_1, avg_err_out_1 = avg_err(Z1, Y)

In [8]:
avg_err_in_1

1069.5699417882126

In [9]:
avg_err_out_1

1070.6328973425902